In [34]:
#read datas
import pandas as pd
dir_data_all = '/Users/apple/Desktop/data/sentiment-analysis-on-movie-reviews/train.tsv'
data_all = pd.read_csv(dir_data_all, sep='\t')
print(data_all.shape)
print(data_all.keys())

(156060, 4)
Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')


In [35]:
#split training datasets, testing datasets and evaluating datasets
#make sure that the size of testing datasets is equal to that of evaluating datasets
from sklearn.model_selection import train_test_split
x_all = data_all['Phrase']
y_all = data_all['Sentiment']
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25)
print(x_train.shape)
print(x_test.shape) #PhraseId Phrase

(93636,)
(31212,)


In [36]:
#get the corpus matrix
#return a tuple(Index of Phrase, Index of representation of the word in the dictionary)
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)
x_test_counts = count_vect.transform(x_test)
print(x_train_counts.shape)
print(x_test_counts.shape)

(93636, 15181)
(31212, 15181)


In [37]:
#word-level tfidf
#use TF to evaluate the frequency of a word appearing in one article(sparse)
#TF = the frequency of a particular word / the whole word in the passage
#searching word is more important
#the more it occurs in the passage, the more important it is
#when there are many conjuctions, it is difficult to distinguish between them
#use IDF to penalize those occuring too much in the passage(how, to, what),reconcling TF
#IDF = log(number of all passages / (number of passages containing the word + 1))
#if one word occurs too much in all passages, it must carry less information
#TF-IDF = TF * IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_transformer = TfidfVectorizer(analyzer='word', max_features=50000)
tfidf_transformer.fit(x_train)
x_train_tfidf_word = tfidf_transformer.transform(x_train)
x_test_tfidf_word = tfidf_transformer.transform(x_test)
print(x_train_tfidf_word.shape)
print(x_test_tfidf_word.shape)

(93636, 15181)
(31212, 15181)


In [38]:
#ngram-level tfidf
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfVectorizer(analyzer='word', ngram_range=(2, 3), max_features=50000)
tfidf_transformer.fit(x_train)
x_train_tfidf_ngram = tfidf_transformer.transform(x_train)
x_test_tfidf_ngram = tfidf_transformer.transform(x_test)
print(x_train_tfidf_ngram.shape, x_test_tfidf_ngram.shape)

(93636, 50000) (31212, 50000)


In [39]:
from scipy.sparse import hstack
train_features=x_train_counts
test_features=x_test_counts

train_features=hstack([x_train_counts, x_train_tfidf_word, x_train_tfidf_ngram])
test_features=hstack([x_test_counts, x_test_tfidf_word, x_test_tfidf_ngram])
print(train_features.shape)

(93636, 80362)


In [40]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(
    random_state=0,
    solver='sag',
    multi_class='multinomial'
)

In [41]:
from sklearn.utils import shuffle
train_features, y_train = shuffle(train_features, y_train)
clf.fit(train_features, y_train)

/Users/apple/opt/anaconda3/envs/mathcup/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(multi_class='multinomial', random_state=0, solver='sag')

In [42]:
import numpy as np
predict = clf.predict(test_features)
print(np.mean(predict==y_test))

0.6539792387543253
